注意力机制
===

# 1.注意力
我们观察事务时，之所以能够快速判断一种事物(允许判断是错误的)，是因为我们大脑能够很快吧注意力放在事物最具有辨识度的部分从而做出判断，而并非是从头到尾的观察一遍事物后，才能有判断结果。于是就产生了注意力机制

# 2.注意力计算规则
它需要三个指定的输入$Q(query),K(key),V(value)$，然后通过计算公式得到注意力的结果，这个结果代表query在key和value作用下的注意力表示，当输入的$Q=K=V$时，称作自注意力计算规则

1. 将$Q,K$进行纵轴拼接，做一次线性变化，在使用softmax处理获得结果最后与$V$做张量乘法
$$Attention(Q,K,V)=Softmax[Linear(Q, K)] \bullet V$$

2. 将$Q,K$进行纵轴拼接，做一次线性变化后再使用tanh函数激活，然后在进行内部求和，最后使用softmax处理获得结果在与$V$做张量乘法
$$Attention(Q,K,V)=Softmax[sum(tanh(Linear(Q, K)))] \bullet V$$

3. 将$Q$与$K^T$作点积运算，然后除以一个缩放系数，在使用softmax处理获得结果最后与$V$作张量乘法
$$Attention(Q,K,V)=softmax(\frac{Q \bullet K^T}{\sqrt{d_k}}) \bullet V$$

当注意力权重矩阵和$V$都是三维张量且第一为代表为batch条数时，则做BMM运算。BMM是一种特殊的张量乘法运算.如果参数1形式是$b \times n \times m$，参数2形状是$b \times m \times p$，则输出为$b \times n \times p$

In [1]:
import torch
input = torch.randn(10, 3, 4)
mat2 = torch.randn(10, 4, 5)
res = torch.bmm(input, mat2)
res.size()

torch.Size([10, 3, 5])

# 3.注意力机制

## 3.1.注意力机制
注意力机制是注意力计算规则能够应用的深度学习网络的载体，同时包括一些必要的全连接层以及相关张量处理，使其与应用网络融为一体，使自注意力计算规则的注意力机制成为自注意力机制。在NLP领域中，当前的注意力机制大多数应用于seq2seq架构，即编码器和解码器。

# 3.2.注意力机制的作用
- 在加码器端的注意力机制：能够根据模型目标有效的聚焦编码器的输出结果，当其作为解码器的输入时提升效果，改善以往编码器输出是单一定长张量，无法存储过多信息的情况
- 在编码器端的注意力机制：主要解决表征问题，相当于特征提取过程，得到输入的注意力表示。一般使用自注意力(self-attention)

## 3.3.注意力机制的实现步骤
1. 根据注意力计算规则，对$Q, K, V$进行相应的计算
2. 根据第一步采用的计算方法，如果是拼接方法，则需要将$Q$与第二步的计算结果在进行拼接，如果是转置点积，一般是自注意力，$Q$与$V$相同，则不需要进行与$Q$的拼接
3. 最后为了使整个attention机制按照指定尺寸输出，使用线性层作用在第二步的结果上做一个线性变换，得到最终对$Q$的注意力表示

# 4.注意力机制的代码分析

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Attn(nn.Module):
    def __init__(self, query_size, key_size, value_size1, value_size2, output_size):
        # query_size: query的最后一维大小
        # key_size: key的最后一维大小
        # value_size1: value的倒数第二维大小，即value = (1, value_size1, value_size2)
        # value_size1: value的倒数第一维大小
        # output_size：输出的最后一维大小
        super(Attn, self).__init__()
        self.query_size = query_size
        self.key_size = key_size
        self.value_size1 = value_size1
        self.value_size2 = value_size2
        self.output_size = output_size

        self.attn = nn.Linear(self.query_size + self.key_size, value_size1)
        self.attn_combine = nn.Linear(self.query_size + value_size2, output_size)
    
    def forward(self, Q, K, V):
        # 输入给Attention机制的张量一般情况都是三维张量，因此这里也假设Q,K,V都是三维张量
        #
        # 第一步，采用常见的第一种计算规则， 将Q,K进行纵轴拼接，做一次线性变化，最后使用softmax处理获得结果
        attn_weights = F.softmax(self.attn(torch.cat((Q[0], K[0]), 1)), dim=1) # 这是注意力权重张量，它是一个二维张量

        # 然后进行第一步的后半部分，将得到的权重矩阵与V做矩阵乘法计算，当两者都是三维张量且第一维代表为batch条数时，则做bmm运算
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), V) # V的大小就是1, value_size1, value_size2, attn_applied是一个三维张量

        # 之后进行第二步，通过去[0]是用来降维，根据第一步采用的计算方法，需要将Q与第一步的计算结果在进行拼接
        output = torch.cat((Q[0], attn_applied[0]), 1)

        # 最后是第三步，使用线性层作用在第三步的结果上做一个线性变换并扩展维度，得到输出，因为要保证输出也是三维张量，
        # 因此使用unsqueeze(0)进行维度扩展
        output = self.attn_combine(output).unsqueeze(0)
        return output, attn_weights

In [4]:
query_size = 32
key_size = 32
value_size1 = 32
value_size2 = 64
output_size = 64

attn = Attn(query_size=query_size, key_size=key_size, value_size1=value_size1, value_size2=value_size2, output_size=output_size)
Q = torch.randn(1, 1, 32)
K = torch.randn(1, 1, 32)
V = torch.randn(1, 32, 64)

output = attn(Q, K, V)
output[0]

tensor([[[-0.4399,  0.4871, -0.1396, -0.1390,  0.3672,  0.4623,  0.1721,
          -0.0945,  0.5629, -0.1152,  0.2424, -0.3700, -0.4103,  0.1828,
           0.2574,  0.5023,  0.2950, -0.5676, -0.1641, -0.0494,  0.1146,
           0.4823,  0.0570,  0.5858, -0.4735, -0.0669,  0.5568, -0.1398,
          -0.5314, -0.2958,  0.2642,  0.3573,  0.1686,  0.7940,  0.1624,
           0.6131,  0.6792,  0.0842,  0.3780, -0.0391,  1.0272,  0.0234,
          -0.4176,  0.1985, -0.2669, -0.5747, -0.3799,  0.0559,  0.0995,
           0.2358, -0.1298,  0.0664,  0.0722, -0.0899,  0.2494,  0.2927,
          -0.0589, -0.2452,  0.1702,  0.5210, -0.5306, -0.0567, -0.1449,
          -0.5620]]], grad_fn=<UnsqueezeBackward0>)

In [5]:
output[1]

tensor([[0.0181, 0.0094, 0.0086, 0.0114, 0.0318, 0.0660, 0.0079, 0.0051, 0.0292,
         0.0682, 0.0372, 0.0250, 0.0126, 0.0369, 0.0486, 0.1290, 0.0280, 0.0195,
         0.0195, 0.0131, 0.0409, 0.0077, 0.0424, 0.0303, 0.0632, 0.0444, 0.0353,
         0.0272, 0.0188, 0.0297, 0.0273, 0.0075]], grad_fn=<SoftmaxBackward>)

In [8]:
output[0].size()

torch.Size([1, 1, 64])

In [9]:
output[1].size()

torch.Size([1, 32])